In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# CD B4 extracting
%cd /content/drive/MyDrive/AIProject/

/content/drive/MyDrive/AIProject


In [ ]:
# To extract the zip file
import zipfile
import os

# Create the target directory
os.makedirs('/content/drive/MyDrive/AIProject/', exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')  # Extract to the specified folder


In [ ]:
# CD into your-repo to work on the CNN files
%cd /content/drive/MyDrive/AIProject/AI-Anime-Girl-Cat-Classifier

/content/drive/MyDrive/AIProject/AI-Anime-Girl-Cat-Classifier


In [ ]:
# transfer images to testing folder
import random
import shutil

# Create the target directories
base_dir = '/content/dataset'
training_dir = os.path.join(base_dir, 'training/')
testing_dir = os.path.join(base_dir, 'testing/')
anime_girls_dir = os.path.join(training_dir, 'anime_girls')
cats_dir = os.path.join(training_dir, 'cats')

os.makedirs(testing_dir, exist_ok=True)
os.makedirs(os.path.join(testing_dir, 'anime_girls'), exist_ok=True)
os.makedirs(os.path.join(testing_dir, 'cats'), exist_ok=True)

# Function to move 20% of images to the testing folder
def move_random_images(src_folder, dest_folder, percentage=0.2):
    # Gets files in the source directory
    all_files = os.listdir(src_folder)

    num_to_move = int(len(all_files) * percentage)

    files_to_move = random.sample(all_files, num_to_move)

    # Move to the destination directory
    for file_name in files_to_move:
        shutil.move(os.path.join(src_folder, file_name), os.path.join(dest_folder, file_name))

move_random_images(anime_girls_dir, os.path.join(testing_dir, 'anime_girls'))
move_random_images(cats_dir, os.path.join(testing_dir, 'cats'))

print("Randomly selected 20% of images moved to the testing folder.")


Randomly selected 20% of images moved to the testing folder.


In [ ]:
import os
import logging
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from PIL import Image


In [ ]:
# Enable logging
logging.basicConfig(filename='image_loading_errors.log', level=logging.ERROR)

# Function to safely load images
def safe_load_img(path):
    try:
        img = Image.open(path)
        img.verify()
        img = img.convert('RGB')
        return np.array(img)
    except Exception as e:
        logging.error(f"Failed to load image: {path} due to {str(e)}")
        print(f"Error loading image: {path}")  # Output the name of the image that caused the error
        return None

# Define image size and batch size
IMG_SIZE = (128, 128)  # Resize images to 128x128
BATCH_SIZE = 32

# Count total images in a directory
def count_images_in_directory(directory):
    count = 0
    for _, _, files in os.walk(directory):
        count += len([f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    return count

# Custom ImageDataGenerator to handle errors
class CustomImageDataGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory, **kwargs):
        generator = super().flow_from_directory(directory, **kwargs)

        while True:
            x_batch, y_batch = next(generator)
            valid_data = []
            for x, y in zip(x_batch, y_batch):
                try:
                    img = Image.fromarray((x * 255).astype(np.uint8))
                    img.verify()
                    valid_data.append((x, y))
                except Exception as e:
                    logging.error(f"Failed to process image in batch due to {str(e)}")
                    print(f"Error processing image in batch.")
            if valid_data:
                x_batch, y_batch = zip(*valid_data)
                yield np.array(x_batch), np.array(y_batch)
            else:
                print("No valid images found in the batch.")

# ImageDataGenerator for loading and augmenting data
train_datagen = CustomImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to [0, 1]
    rotation_range=20,         # Randomly rotate images
    width_shift_range=0.2,     # Randomly shift images horizontally
    height_shift_range=0.2,    # Randomly shift images vertically
    shear_range=0.2,           # Shear transformation
    zoom_range=0.2,            # Random zoom
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill missing pixels after transformations
)

validation_datagen = CustomImageDataGenerator(rescale=1./255)

# Directories
train_dir = '/content//dataset/training'
validation_dir = '/content/dataset/testing'

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification (anime girls vs. cats)
    color_mode='rgb'
)

# Load validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb'
)

# Calculate total samples manually for the training and validation generators
total_train_samples = count_images_in_directory(train_dir)
total_validation_samples = count_images_in_directory(validation_dir)

model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary output

# Compiler for the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# Train the model
try:
    history = model.fit(
        train_generator,
        steps_per_epoch=total_train_samples // BATCH_SIZE,
        epochs=15,
        validation_data=validation_generator,
        validation_steps=total_validation_samples // BATCH_SIZE
    )
except Exception as e:
    print(f"Training interrupted: {str(e)}")

model.save('anime_girls_vs_cats_cnn.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,601,153 (9.92 MB)

 Trainable params: 2,601,153 (9.92 MB)

 Non-trainable params: 0 (0.00 B)

Found 55187 images belonging to 2 classes.
Epoch 1/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9781 - loss: 0.0639Found 13796 images belonging to 2 classes.
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 2420s 1s/step - accuracy: 0.9782 - loss: 0.0638 - val_accuracy: 0.9975 - val_loss: 0.0081
Epoch 2/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 2369s 1s/step - accuracy: 0.9943 - loss: 0.0170 - val_accuracy: 0.9975 - val_loss: 0.0082
Epoch 3/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 2355s 1s/step - accuracy: 0.9959 - loss: 0.0134 - val_accuracy: 0.9974 - val_loss: 0.0114
Epoch 4/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 2305s 1s/step - accuracy: 0.9967 - loss: 0.0109 - val_accuracy: 0.9993 - val_loss: 0.0024
Epoch 5/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 2346s 1s/step - accuracy: 0.9967 - loss: 0.0109 - val_accuracy: 0.9988 - val_loss: 0.0032
Epoch 6/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 2323s 1s/step - accuracy: 0.9968 - loss: 0.0098 - val_accuracy: 0.9984 - val_loss: 0.0045
Epoch 7/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 2335s

In [28]:
import os
from google.colab import files
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from PIL import Image

# Load model
model = load_model('/content/drive/MyDrive/AIProject/AI-Anime-Girl-Cat-Classifier/anime_girls_vs_cats_cnn.h5')

# Directory where uploaded images will be saved
user_images_dir = '/content/drive/MyDrive/AIProject/AI-Anime-Girl-Cat-Classifier/user test images'

# Create the directory if it doesn't exist
os.makedirs(user_images_dir, exist_ok=True)

def classify_image(img):
    # Resize and normalize the image
    img = img.resize((128, 128))
    img = np.array(img) / 255.0

    # Check if the image has 3 channels (RGB)
    if len(img.shape) != 3 or img.shape[2] != 3:
        # Convert to RGB if not
        img = img.convert('RGB')
        img = np.array(img) / 255.0

    # Add batch dimension
    img = np.expand_dims(img, axis=0)

    # Predict the class of the image
    prediction = model.predict(img)

    # Interpret the result
    if prediction[0][0] <= 0.5:
        return "It's an Anime Girl!"
    else:
        return "It's a Cat!"

# Keep running until the user decides to stop
while True:
    print("Upload an image to classify or type 'stop' to exit:")

    # Step 1: Upload the image
    uploaded = files.upload()

    if 'stop' in uploaded:
        print("Stopping the loop.")
        break

    # Step 2: Save and classify each uploaded image
    for fn in uploaded.keys():
        file_path = os.path.join(user_images_dir, fn)
        with open(file_path, 'wb') as f:
            f.write(uploaded[fn])

        # Load the image from the saved location and classify it
        img = Image.open(file_path)
        result = classify_image(img)

        print(f"Result for {fn}: {result}")


Upload an image to classify or type 'stop' to exit:


KeyboardInterrupt: 